# NHANES I Survival Model

This is a cox proportional hazards model on data from <a href="https://wwwn.cdc.gov/nchs/nhanes/nhanes1">NHANES I</a> with followup mortality data from the <a href="https://wwwn.cdc.gov/nchs/nhanes/nhefs">NHANES I Epidemiologic Followup Study</a>. It is designed to illustrate how SHAP values enable the interpretion of XGBoost models with a clarity traditionally only provided by linear models. We see interesting and non-linear patterns in the data, which suggest the potential of this approach. Keep in mind the data has not yet been checked by us for calibrations to current lab tests and so you should not consider the results as actionable medical insights, but rather a proof of concept. 

Note that support for Cox loss and SHAP interaction effects were only recently merged, so you will need the latest master version of XGBoost to run this notebook.

In [4]:
import shap
import xgboost
from sklearn.model_selection import train_test_split
import matplotlib.pylab as pl
import numpy as np
import pandas as pd

## Create XGBoost data objects

This uses a pre-processed subset of NHANES I data available in the SHAP datasets module.

In [5]:
X,y = shap.datasets.nhanesi()
X_display,y_display = shap.datasets.nhanesi(display=True) # human readable feature values

xgb_full = xgboost.DMatrix(X, label=y)

# create a train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
xgb_train = xgboost.DMatrix(X_train, label=y_train)
xgb_test = xgboost.DMatrix(X_test, label=y_test)

In [6]:
X.head(5)

,Unnamed: 0,Age,Diastolic BP,Poverty index,Race,Red blood cells,Sedimentation rate,Serum Albumin,Serum Cholesterol,Serum Iron,Serum Magnesium,Serum Protein,Sex,Systolic BP,TIBC,TS,White blood cells,BMI,Pulse pressure
0,0,35.0,92.0,126.0,2.0,77.7,12.0,5.0,165.0,135.0,1.37,7.6,2.0,142.0,323.0,41.8,5.8,31.109434,50.0
1,1,71.0,78.0,210.0,2.0,77.7,37.0,4.0,298.0,89.0,1.38,6.4,2.0,156.0,331.0,26.9,5.3,32.362572,78.0
2,2,74.0,86.0,999.0,2.0,77.7,31.0,3.8,222.0,115.0,1.37,7.4,2.0,170.0,299.0,38.5,8.1,25.388497,84.0
3,3,64.0,92.0,385.0,1.0,77.7,30.0,4.3,265.0,94.0,1.97,7.3,2.0,172.0,349.0,26.9,6.7,26.446610,80.0
4,4,32.0,70.0,183.0,2.0,77.7,18.0,5.0,203.0,192.0,1.35,7.3,1.0,128.0,386.0,49.7,8.1,20.354684,58.0


In [7]:
X.shape

(9932, 19)





## Train XGBoost model

In [ ]:
# use validation set to choose # of trees
params = {
    "eta": 0.002,
    "max_depth": 3,
    "objective": "survival:cox",
    "subsample": 0.5
}
model_train = xgboost.train(params, xgb_train, 10000, evals = [(xgb_test, "test")], verbose_eval=1000)

In [8]:
# train final model on the full data set
params = {
    "eta": 0.002,
    "max_depth": 3, 
    "objective": "survival:cox",
    "subsample": 0.5
}
model = xgboost.train(params, xgb_full, 5000, evals = [(xgb_full, "test")], verbose_eval=1000)

[20:24:21] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[0]	test-cox-nloglik:8.88072
[20:24:21] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:21] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:21] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:21] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:21] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:21] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:21] C:\dev\libs\xgboost\src\tree\updater

[20:24:24] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:24] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:24] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:24] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:24] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:24] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:24] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:24] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning en

[20:24:25] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:25] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:25] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:25] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:25] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:25] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:25] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:25] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning en

[20:24:28] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:28] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:28] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:28] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:28] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:28] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:28] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:28] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning en

[20:24:32] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:32] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:32] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:32] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:32] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:32] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:32] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:24:32] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning en

...

[20:28:37] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:37] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:37] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:37] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[20:28:37] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:37] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:37] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[20:28:37] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end

[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning en

[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[20:28:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning en

[20:28:41] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:41] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:41] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:41] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[20:28:41] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:41] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:41] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:41] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning en

[20:28:44] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:44] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[20:28:44] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:44] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:44] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:44] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:44] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[20:28:44] C:\dev\libs\xgboost\src\tree\updater_prune.cc:75: tree pruning en

## Check Performance

The C-statistic measures how well we can order people by their survival time (1.0 is a perfect ordering).

In [9]:
def c_statistic_harrell(pred, labels):
    total = 0
    matches = 0
    for i in range(len(labels)):
        for j in range(len(labels)):
            if labels[j] > 0 and abs(labels[i]) > labels[j]:
                total += 1
                if pred[j] > pred[i]:
                    matches += 1
    return matches/total

# see how well we can order people by survival
c_statistic_harrell(model.predict(xgb_test, ntree_limit=5000), y_test)

0.8524535925513849

## Model Scoring script

In [10]:
def convert(logit):
    odds = np.exp(logit)
    prob = odds / (1 + odds)
    return prob

In [11]:
def score(testdf):
    dtest = xgboost.DMatrix(testdf)
    xgb_preds = model.predict(dtest)
    results = []
    for x in np.nditer(xgb_preds):
        prob = convert(x)
        results.append(prob)
    df = pd.DataFrame({'prediction':results})
    return df

In [12]:
scores = score(X)
scores.head()

,prediction
0,0.539799
1,0.920343
2,0.966797
3,0.644484
4,0.537251


## Explain the model's predictions on the entire dataset